## Model 2: SVMs

In [27]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import RepeatedKFold, cross_validate

from sklearn.naive_bayes import CategoricalNB

### Process Data

In [28]:
df = pd.read_csv('proc.csv', index_col=0)
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))] #Copied off stackoverflow here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe

In [29]:
df['cat_cluster'] = df['cat_cluster'].fillna(0) #Fill NAs with 0s (should be in above section but i forgot :/)
df['location_cluster'] = df['location_cluster'].fillna(0)

In [30]:
str_cols = ['name', 'address', 'gps', 'top_category', 'relevant_cat', 'translated', 'charged_words', 'price_words', 'categories', 'unixReviewTime', 'reviewTime']
df_num = df[df.columns.drop(str_cols)]

In [31]:
df_num.head(5)

,closed,gPlusPlaceId,Class_$,Class_$$,Class_$$$,Class_$$$$,location_cluster,rating,cat_cluster,Class_ca,...,Class_de,Class_en,Class_es,Class_fr,Class_hi,Class_it,Class_ms,Class_nl,Class_pt,Class_tr
0,False,115757957627721988675,0,1,0,0,0.0,5.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
1,False,116555416797255000560,0,1,0,0,0.0,4.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,True,114781865961627441828,0,0,1,0,0.0,5.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,114735143729299990529,0,1,0,0,0.0,5.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,105589668159024738610,0,1,0,0,0.0,5.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
def cost_label_enc(row):
  if row['Class_$'] == 1:
    return 1
  elif row['Class_$$'] == 1:
    return 2
  elif row['Class_$$$'] == 1:
    return 3
  else:
    return 4

In [33]:
df_num['cost'] =df_num.apply(cost_label_enc, axis=1)
df_num.drop(['Class_$', 'Class_$$', 'Class_$$$', 'Class_$$$$'], axis=1, inplace=True)

<ipython-input-33-a9b45a91013f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num['cost'] =df_num.apply(cost_label_enc, axis=1)
<ipython-input-33-a9b45a91013f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num.drop(['Class_$', 'Class_$$', 'Class_$$$', 'Class_$$$$'], axis=1, inplace=True)


In [34]:
X = df_num.drop(['cost'], axis=1).astype(float)
y = df_num[['cost']].astype(float)

In [35]:
#Resample
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=42)
#X_res, y_res = ros.fit_resample(X, y)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [38]:
#Scale all numbers
X_train_num = scaler.fit_transform(X_train)
X_test_num = scaler.fit_transform(X_test)

In [39]:
# import our svm libraries
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [40]:
#svm = SVC(kernel = 'linear', class_weight={1.0: 1, 2.0: 1.1160116448326055, 3.0: 1.9847222222222223, 4.0: 11.484848484848484}, decision_function_shape='ovo')

#param_grid = {'C': [ 10, 100, 1000, 10_000, 100_000],
#              'gamma': [5, 3, 1, 0.1, 0.01, 0.001, 0.0001],
#              'kernel': ['rbf', 'sigmoid'],
#              'class_weight': [{1.0: 1, 2.0: 1.1160116448326055, 3.0: 1.9847222222222223, 4.0: 11.484848484848484}]}

#grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2)

#grid.fit(X_train_num, y_train.values.ravel())

In [41]:
#grid.best_params_

In [47]:
svm = SVC(kernel = 'linear', class_weight={1.0: 1, 2.0: 1.1160116448326054, 3.0: 1.9847222222222223, 4.0: 11.484848484848484}, decision_function_shape='ovo')

In [48]:
svm.fit(X_train_num, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(class_weight={1.0: 1, 2.0: 1.1160116448326054, 3.0: 1.9847222222222223,
                  4.0: 11.484848484848484},
    decision_function_shape='ovo', kernel='linear')

In [49]:
y_pred = svm.predict(X_test_num)

In [50]:
y_pred

array([2., 2., 2., 2., 2., 3., 2., 3., 2., 2., 2., 3., 3., 2., 3., 2., 2.,
       3., 2., 2., 2., 3., 2., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 3., 2., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 3.,
       3., 3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 3., 3., 2., 3., 2.,
       2., 2., 3., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 3., 2., 2., 2.,
       3., 2., 2., 2., 2., 3., 2., 2., 3., 3., 2., 2., 2., 2., 2., 3., 2.,
       2., 2., 2., 3., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3.,
       2., 3., 2., 2., 3., 2., 2., 2., 2., 3., 2., 2., 3., 2., 3., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 2., 2.,
       2., 2., 2., 2., 2., 3., 2., 2.])

In [51]:
#Get results:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         2.0       0.63      0.80      0.70       105
         3.0       0.52      0.33      0.40        70
         4.0       0.00      0.00      0.00         3

    accuracy                           0.60       178
   macro avg       0.38      0.38      0.37       178
weighted avg       0.58      0.60      0.57       178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
#Training error for SVM
y_train_pred = svm.predict(X_train_num)
train_error = [list(y_train.cost)[i] == y_train_pred[i] for i in range(len(y_train_pred))]

In [56]:
np.mean(train_error)

0.6304654442877292

In [58]:
#Testing error
#Training error for SVM
y_test_pred = svm.predict(X_test_num)
test_error = [list(y_test.cost)[i] == y_test_pred[i] for i in range(len(y_test_pred))]

print(np.mean(test_error))

0.601123595505618
